<a href="https://colab.research.google.com/github/abhinav9800/abhinav98/blob/main/image_classification_using_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# dataset - https://www.kaggle.com/datasets/pavansanagapati/images-dataset
# The dataset contains images of cars, cats, flowers, horses, humans, dogs, and bikes.

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d pavansanagapati/images-dataset

 99% 1.01G/1.01G [00:07<00:00, 44.1MB/s]
100% 1.01G/1.01G [00:07<00:00, 138MB/s] 


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/images-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
!mkdir /content/data/train
!mkdir /content/data/test

In [6]:
import os
import random
import shutil

In [7]:
!pip install -U efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.3 MB/s eta 0:00:00


In [8]:
import efficientnet.keras as efn
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.callbacks import EarlyStopping

In [9]:
conv_base = efn.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    classes=7,
    input_shape=(150, 150, 3),
    drop_connect_rate=0.4
)


16804768/16804768 [==============================] - 0s 0us/step


In [10]:
def split_data(data_paths, train_base_folder, test_base_folder, train_ratio=0.85):
    """
    Splits the image datasets into train and test datasets.

    Args:
    - data_paths (dict): A dictionary mapping category names to their respective data paths.
    - train_base_folder (str): The base folder where the train dataset will be stored.
    - test_base_folder (str): The base folder where the test dataset will be stored.
    - train_ratio (float): The ratio of images to be allocated for the training dataset.

    Returns:
    - None
    """

    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    random.seed(42)

    for category, data_path in data_paths.items():

        train_folder = os.path.join(train_base_folder, category)
        test_folder = os.path.join(test_base_folder, category)

        for folder_path in [train_folder, test_folder]:
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

        imgs_list = [filename for filename in os.listdir(data_path) if os.path.splitext(filename)[-1] in image_extensions]

        random.shuffle(imgs_list)

        train_size = int(len(imgs_list) * train_ratio)
        test_size = len(imgs_list) - train_size

        for i, f in enumerate(imgs_list):
            if i < train_size:
                dest_folder = train_folder
            else:
                dest_folder = test_folder
            shutil.copy(os.path.join(data_path, f), os.path.join(dest_folder, f))

# Example usage:
data_paths = {
    "car": "/content/data/cars",
    "cat": "/content/data/cats",
    "flower": "/content/data/flowers",
    "horse": "/content/data/horses",
    "human": "/content/data/human",
    "dog": "/content/data/dogs",
    "bike": "/content/data/bike"
}

train_base_folder = "/content/data/train"
test_base_folder = "/content/data/test"

split_data(data_paths, train_base_folder, test_base_folder)


In [11]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

In [12]:
def augment_images(train_dirs, batch_size=32, target_size=(150, 150), augmentation_parameters=None):
    if augmentation_parameters is None:
        augmentation_parameters = {
            'rescale': 1./255,
            'shear_range': 0.2,
            'zoom_range': 0.2,
            'horizontal_flip': True
        }

    """
    Function to augment images in specified directories using Keras ImageDataGenerator.

    Args:
    - train_dirs (list): List of directories containing the training images.
    - batch_size (int): Batch size for data augmentation.
    - target_size (tuple): Tuple specifying the target size of the images (height, width).
    - augmentation_parameters (dict): Dictionary of augmentation parameters for ImageDataGenerator.
                                      Defaults to None, which uses predefined augmentation parameters.

    Returns:
    - train_generator (DirectoryIterator): Iterator for training data.
    - validation_generator (DirectoryIterator): Iterator for validation data.
    """

    train_datagen = ImageDataGenerator(**augmentation_parameters)

    for data_dir in train_dirs:
        image_files = os.listdir(data_dir)

        for image_file in image_files:
            image_path = os.path.join(data_dir, image_file)
            image = load_img(image_path)
            image_array = img_to_array(image)
            image_array = image_array.reshape((1,) + image_array.shape)

            i = 0
            for batch in train_datagen.flow(image_array,
                                             batch_size=1,
                                             save_to_dir=data_dir,
                                             save_prefix='augmented_' + image_file.split('.')[0],
                                             save_format='jpg'):
                i += 1
                if i >= 2:
                    break

    train_generator = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

    test_datagen = ImageDataGenerator(rescale=1./255)

    validation_generator = test_datagen.flow_from_directory(
        '/content/data/test',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')

    return train_generator, validation_generator

# Example usage
train_dirs = ["/content/data/train/bike",
              "/content/data/train/car",
              "/content/data/train/cat",
              "/content/data/train/flower",
              "/content/data/train/horse",
              "/content/data/train/human",
              "/content/data/train/dog"]

train_generator, validation_generator = augment_images(train_dirs, batch_size=32, target_size=(150, 150))


Found 4576 images belonging to 7 classes.
Found 274 images belonging to 7 classes.


In [13]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [14]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/data/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/data/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150)
)

Found 4576 files belonging to 7 classes.
Found 274 files belonging to 7 classes.


In [15]:
def process(image, label):
    image = tf.cast(image / 255., tf.float32)
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [16]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=2)

history = model.fit(train_ds, epochs=10, validation_data=validation_ds,callbacks=[early_stopping_callback])


Epoch 1/10
143/143 [==============================] - 51s 160ms/step - loss: 0.7538 - accuracy: 0.7533 - val_loss: 0.2662 - val_accuracy: 0.9161
Epoch 2/10
143/143 [==============================] - 20s 134ms/step - loss: 0.1479 - accuracy: 0.9613 - val_loss: 0.1501 - val_accuracy: 0.9599
Epoch 3/10
143/143 [==============================] - 20s 135ms/step - loss: 0.0574 - accuracy: 0.9869 - val_loss: 0.1177 - val_accuracy: 0.9708
Epoch 4/10
143/143 [==============================] - 19s 133ms/step - loss: 0.0304 - accuracy: 0.9921 - val_loss: 0.0990 - val_accuracy: 0.9745
Epoch 5/10
143/143 [==============================] - 19s 133ms/step - loss: 0.0207 - accuracy: 0.9952 - val_loss: 0.1065 - val_accuracy: 0.9745
Epoch 6/10
143/143 [==============================] - 20s 138ms/step - loss: 0.0137 - accuracy: 0.9974 - val_loss: 0.1020 - val_accuracy: 0.9745
